In [1]:
#Import modules
import pandas as pd
import numpy as np

#Importcsv file to a dataframe
df = pd.read_csv('C:/Users/thima/Desktop/sahil/reviews.csv')

In [2]:
# subset to get data for location_id 4962_201 and removing missing values
dfa = df[df['location_id'] == '4962_201'].dropna()
# Extract rating clumn for this location
loc4962_201 = dfa['rating']

In [3]:
# subset to get data for location_id 4962_380 and removing missing values
dfb = df[df['location_id'] == '4962_380'].dropna()
# Extract rating clumn for this location
loc4962_380 = dfb['rating']

In [4]:
#Get mean for location_id 4962_201
loc4962_201.mean()

4.750539956803456

In [5]:
#Get mean for location_id 4962_380
loc4962_380.mean()

4.669796557120501

In [6]:
# F test - test for equality of variance
from scipy import stats
stats.levene(loc4962_201, loc4962_380)

LeveneResult(statistic=2.9216214439327843, pvalue=0.08759850318285413)

In [7]:
#t test

results = stats.ttest_ind(loc4962_201, loc4962_380)
alpha = 0.05
if (results[0] > 0) & (results[1]/2 < alpha):
    print ("reject null hypothesis")
else:
    print ("do not reject null hypothesis")
# reject null hypothesis, mean of loc4962_201 is greater than mean of loc4962_380

reject null hypothesis


In [9]:
# subset to get data for location_id 4962_381 and removing missing values
dfc = df[df['location_id'] == '4962_381'].dropna()
# Extract rating clumn for this location
loc4962_381 = dfc['rating']

In [10]:
# subset to get data for location_id 4962_915 and removing missing values
dfd = df[df['location_id'] == '4962_915'].dropna()
# Extract rating clumn for this location
loc4962_915 = dfd['rating']

In [23]:
#Get mean for location_id 4962_381
loc4962_381.mean()


4.554561717352415

In [24]:
#Get mean for location_id 4962_915
loc4962_915.mean()

4.629558541266794

In [16]:
# F test - test for equality of variance
stats.levene(loc4962_381, loc4962_915)

LeveneResult(statistic=1.3001425578191435, pvalue=0.25444045416051253)

In [20]:
#t test
stats.ttest_ind(loc4962_381, loc4962_915)
# do  not reject null hypothesis, equal means

Ttest_indResult(statistic=-1.1402379391246042, pvalue=0.2544404541606672)

In [48]:
#Extract year from data
df['year'] = pd.DatetimeIndex(df['date']).year

In [49]:
#create dummy variables for source
cat_vars=['source']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    df1=df.join(cat_list)
    df=df1
cat_vars=['source']
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
dfn=df[to_keep]
dfn.columns.values


array(['location_id', 'review_id', 'date', 'rating', 'year',
       'source_CARS', 'source_FACEBOOK', 'source_GOOGLE_PLACES'],
      dtype=object)

In [50]:
#remove missing values
dfn = dfn[pd.notnull(dfn['rating'])]

In [51]:
#rating count
dfn['rating'].value_counts()

5.0    139838
1.0     18192
4.0     13636
3.0      4171
2.0      3439
Name: rating, dtype: int64

In [54]:
#predictors
X = dfn[['location_id', 'source_CARS', 'source_FACEBOOK', 'source_GOOGLE_PLACES',
       'year']]


In [55]:
#output variable
Y=dfn['rating']

In [56]:
#train(70%) test(30%) split
from sklearn.model_selection import train_test_split
X_sample, X_test, y_sample, y_test = train_test_split(X,Y,test_size=0.3,random_state=101)

In [57]:
#SMOTE sampling training data
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE().fit_sample(X_sample, y_sample)

In [58]:
# Resampled dataset shape
from collections import Counter
print('Resampled dataset shape %s' % Counter(y_train))

Resampled dataset shape Counter({5.0: 97954, 3.0: 97954, 1.0: 97954, 4.0: 97954, 2.0: 97954})


In [61]:
# Model Selection
#logistic regression

#Model bulding and prediction
from sklearn import linear_model
gb2 = linear_model.LogisticRegressionCV()
gb2.fit(X_train, y_train)

#train test accuracy
print("Accuracy of train set: {:.3f}".format(gb2.score(X_train, y_train)))
print("Accuracy of test set: {:.3f}".format(gb2.score(X_test, y_test)))

Accuracy of train set: 0.198
Accuracy of test set: 0.024


In [60]:
#performance metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
predictions = gb2.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))
#We leave logistic regression as we have low precision, recalll and F1 score. Also, lower accuracy

Confusion Matrix:
[[    0   457  5003    34     0]
 [    0    88  1000     5     0]
 [    0   123  1152    13     0]
 [    0   331  3645    48     0]
 [    0  3393 38179   312     0]]

Classification Report
             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00      5494
        2.0       0.02      0.08      0.03      1093
        3.0       0.02      0.89      0.05      1288
        4.0       0.12      0.01      0.02      4024
        5.0       0.00      0.00      0.00     41884

avg / total       0.01      0.02      0.00     53783



C:\Users\thima\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
# Model Selection
#Decision Tree

#Model bulding and prediction
 
from sklearn import tree
gb3 = tree.DecisionTreeClassifier(max_depth=3)
gb3.fit(X_train, y_train)
#forest.fit(X_regtrain, y_regtrain)
print("Accuracy of train set: {:.3f}".format(gb3.score(X_train, y_train)))
print("Accuracy of test set: {:.3f}".format(gb3.score(X_test, y_test)))

Accuracy of train set: 0.299
Accuracy of test set: 0.257


In [63]:
#performance metrics
predictions = gb3.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))
# we get decent results, but we further explore other models

Confusion Matrix:
[[ 1355     0  3757     0   382]
 [  174     0   833     0    86]
 [  151     0  1051     0    86]
 [  407     0  2763     0   854]
 [ 5905     0 24560     0 11419]]

Classification Report
             precision    recall  f1-score   support

        1.0       0.17      0.25      0.20      5494
        2.0       0.00      0.00      0.00      1093
        3.0       0.03      0.82      0.06      1288
        4.0       0.00      0.00      0.00      4024
        5.0       0.89      0.27      0.42     41884

avg / total       0.71      0.26      0.35     53783



C:\Users\thima\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
#Gradient Boosting

#gb = GradientBoostingClassifier(verbose=True)
#param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
#              'max_depth': [4, 6, 8],
#              'min_samples_leaf': [20, 50,100,150],
#              'max_features': [1.0, 0.3, 0.1],
#              'n_estimators':[70,100,120,150]} 
#grid_search = GridSearchCV(gb, param_grid, cv=5)
#grid_search.fit(X_train, y_train)
#print("Best parameters: {}".format(grid_search.best_params_))
#print("Best cross-validation score {:.2f}".format(grid_search.best_score_))

In [65]:
# Model Selection
#Gradient boosting

#Model bulding and prediction
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
gb = GradientBoostingClassifier(learning_rate=0.1,max_depth=6,min_samples_leaf=15,
        max_features=0.3,n_estimators=100,verbose=True)
gb.fit(X_train, y_train)
#forest.fit(X_regtrain, y_regtrain)
print("Accuracy of train set: {:.3f}".format(gb.score(X_train, y_train)))
print("Accuracy of test set: {:.3f}".format(gb.score(X_test, y_test)))

C:\Users\thima\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\thima\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\thima\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


      Iter       Train Loss   Remaining Time 
         1      776923.2620            4.74m
         2      767461.6235            4.60m
         3      759031.3518            4.74m
         4      752873.2085            4.61m
         5      747251.8348            4.47m
         6      741685.2245            4.43m
         7      736513.1152            4.42m
         8      731845.4850            4.46m
         9      728927.1701            4.44m
        10      725538.0921            4.38m
        20      703212.3558            3.95m
        30      690902.5484            3.44m
        40      683610.6981            2.91m
        50      677072.4343            2.45m
        60      670576.3774            1.99m
        70      666202.3610            1.49m
        80      661863.2124           59.63s
        90      658204.6877           29.99s
       100      654603.5666            0.00s
Accuracy of train set: 0.406
Accuracy of test set: 0.371


In [66]:
#performance metrics
predictions = gb.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))
print()
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix:
[[ 2656    50  1265   597   926]
 [  406    13   323   161   190]
 [  341    17   470   255   205]
 [  987    34  1144   763  1096]
 [10671   347  8812  5988 16066]]

Classification Report
             precision    recall  f1-score   support

        1.0       0.18      0.48      0.26      5494
        2.0       0.03      0.01      0.02      1093
        3.0       0.04      0.36      0.07      1288
        4.0       0.10      0.19      0.13      4024
        5.0       0.87      0.38      0.53     41884

avg / total       0.70      0.37      0.45     53783



In [67]:
#Load test reviews file
dft = pd.read_csv('C:/Users/thima/Desktop/sahil/test_reviews.csv')
#data preparation
dft = dft.drop(['rating'], axis=1)
dft = dft[pd.notnull(dft['location_id'])]

In [68]:
#create data copy
dftest = dft.copy()

In [69]:
#extract year from data
dftest['year'] = pd.DatetimeIndex(dftest['date']).year

#create dummy for source column
cat_vars=['source']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(dftest[var], prefix=var)
    dfxx=dftest.join(cat_list)
    dftest=dfxx
cat_vars=['source']
data_vars=dftest.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
dfzz=dftest[to_keep]
dfzz.columns.values

array(['location_id', 'review_id', 'date', 'year', 'source_CARS',
       'source_FACEBOOK', 'source_GOOGLE_PLACES'], dtype=object)

In [70]:
#predictors
Xtest = dfzz[['location_id', 'source_CARS', 'source_FACEBOOK', 'source_GOOGLE_PLACES',
       'year']]

In [72]:
predictions2 = gb.predict(Xtest)

In [73]:
dft['rating'] = predictions2

In [74]:
dft

,location_id,review_id,source,date,rating
0,4962_147,test_10368849285,FACEBOOK,12/4/15,1.0
1,4962_147,test_1042394214,FACEBOOK,11/29/16,1.0
2,4962_185,test_11322162891,GOOGLE_PLACES,8/3/16,1.0
3,4962_185,test_11324660949,GOOGLE_PLACES,2/3/18,4.0
4,4962_185,test_11348252095,GOOGLE_PLACES,2/12/16,1.0
5,4962_185,test_1137850456,GOOGLE_PLACES,2/12/18,4.0
6,4962_185,test_1142750998,GOOGLE_PLACES,10/15/15,1.0
7,4962_185,test_11459350091,GOOGLE_PLACES,2/15/18,4.0
8,4962_185,test_11485163368,GOOGLE_PLACES,9/15/17,3.0
9,4962_185,test_11486603498,CARS,11/4/17,5.0


In [75]:
#dataframe to csv
dft.to_csv(r'C:\Users\thima\Desktop\sahil\testreviews.csv', index = None, header=True)